# DEPENDICIES

In [1]:
##DEPENDICIES
import torch
import torchvision
import torch.nn as nn
import numpy as np, os, sys, joblib
import matplotlib.pyplot as pl
import pandas as pd
import random, os
import librosa
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
 #from torch.utils.data import random_split
from torch.optim import lr_scheduler
import time
import tqdm
from evaluate_model import *
from my_helper_code import *
from helper_code import *
from model import *
%matplotlib inline

ModuleNotFoundError: No module named 'evaluate_model'

In [2]:
a = torch.tensor((64,320,8))
b = torch.tensor((64,320,8))
torch.cov(a)

tensor(27669.3340)

In [3]:

def seed_everything(seed: int):
    
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(0)

# data accusation

In [4]:
df = pd.read_csv('dx_mapping_scored.csv')
labels = df['SNOMEDCTCode'].values
labels = [str(i) for i in labels]
classes = list(labels)
train_data_directory = 'training_data'
train_header_files, train_recording_files = find_challenge_files(train_data_directory)
train_num_recordings = len(train_recording_files)
twelve_leads = ('I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6')
six_leads = ('I', 'II', 'III', 'aVR', 'aVL', 'aVF')
four_leads = ('I', 'II', 'III', 'V2')
three_leads = ('I', 'II', 'V2')
two_leads = ('I', 'II')
lead_sets = (twelve_leads, six_leads, four_leads, three_leads, two_leads)

test_data_directory = 'test_data'
test_header_files, test_recording_files = find_challenge_files(test_data_directory)
test_num_recordings = len(test_recording_files)
training_classes = list(labels)
test_classes = list(labels)
num_classes = len(classes)
num_classes

30

# Dataset

In [5]:
class My_Dataset_separate(Dataset) :
    def __init__(self,header_files, recording_files, leads, sample_length, sample_rate = 500) :
        super(Dataset,self).__init__()
        self.header_files = header_files
        self.recording_files = recording_files
        self.leads = leads
        self.sample_length = sample_length    
        self.sample_rate = sample_rate
    def __len__(self) :
         
        return len(self.recording_files)
    
    def __getitem__(self,index) :
        header = load_header(self.header_files[index])
        orig_sr = int(header.split(' ')[2])
        #print(orig_sr)
        recording = load_recording(self.recording_files[index])
        recordings = choose_leads(recording, header, self.leads)
        data = np.zeros((recordings.shape[0],self.sample_length))
        for i in range(len(recordings)):
            #print(type(data[i]))
            y = librosa.resample(recordings[i].astype(np.float), orig_sr, self.sample_rate, res_type='kaiser_best') 
            #print(y.shape[0])
            if y.shape[0] < self.sample_length :
                
                data[i,0:y.shape[0]] = y
            elif y.shape[0] >= self.sample_length:
                data[i] = y[0:self.sample_length]
        current_labels = get_labels(header)
        #print(current_labels)
        labels = np.zeros(( num_classes))
        for label in current_labels:
            if label in classes:
                j = classes.index(label)
                labels[j] = 1
#         data =recordings[:,0:self.sample_length]
       #data = data*10/np.linalg.norm(data)
#         for i in range(len(data)):
#             data[i] = data[i]/max(abs(data[i]))
#         orig_sr = int(header.split(' ')[2])
#         for i in range(len(data)):
#             #print(type(data[i]))
#             data[i] = librosa.resample(data[i].astype(np.float), orig_sr, self.sample_rate, res_type='kaiser_best')        
        recording_id = get_recording_id(header)
        if data.shape[1] < self.sample_length :
            print(data)
        return (data,labels,self.header_files[index])

In [6]:
# sample_length = 4096
# data = My_Dataset_separate(train_header_files, train_recording_files, twelve_leads,sample_length)
# a,b,c =data[50]
# print('-----------------------data----------------')
# print(a)
# print('----------------------data shape -----------')
# print(a.shape)
# print('---------------------- labels -----------')
# print(b)
# print('----------------------labels shape -----------')
# print(b.shape)
# print('----------------------index -----------')
# print(c)

# Dataloader

In [7]:
sample_length = 4096
BATCH_SIZE  = 128
train_dataset = My_Dataset_separate(train_header_files, train_recording_files, twelve_leads,sample_length)
test_dataset = My_Dataset_separate(test_header_files, test_recording_files, twelve_leads,sample_length)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, num_workers=0, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1,
                             shuffle=False, num_workers=0, pin_memory=True)

# Model v.0.0

In [8]:
# class Inception_module(nn.Module) :
#     def __init__(self,n_filters_in, n_filters_out, activision_function = 'relu'):
#         super(Inception_module,self).__init__()
        
#         self.n_filters_in = n_filters_in
#         self.num_features = n_filters_out
#         self.n_filters_out = int(n_filters_out/4)
#         self.activision_function = activision_function
#         self.conv_1d_1 = nn.Conv1d(self.n_filters_in, self.n_filters_out, kernel_size=1)
#         self.conv_1d_7 = nn.Conv1d(self.n_filters_in, self.n_filters_out, kernel_size=7, padding = 3)
#         self.conv_1d_17 = nn.Conv1d(self.n_filters_in, self.n_filters_out, kernel_size=17, padding = 8)
#         self.conv_1d_1_2 = nn.Conv1d(self.n_filters_in, self.n_filters_out, kernel_size=1, padding = 0)
#         self.conv_1d_1_3 = nn.Conv1d(self.n_filters_in, self.n_filters_out, kernel_size=1, padding = 0)
#         self.conv_1d_1_4 = nn.Conv1d(self.n_filters_in, self.n_filters_out, kernel_size=1, padding = 0)
#         self.maxpool_1d = nn.MaxPool1d(7, stride=1)
#         self.batch_norm_1d = nn.BatchNorm1d(num_features = self.num_features )
            
            
#     def forward(self, x) :
        
#         x_1 = self.conv_1d_1(x)
#         #print(x_1.shape)
#         x_2 = self.conv_1d_1_2(x)
#         x_2 = self.conv_1d_7(x)
#         #print(x_2.shape)
#         x_3 = self.conv_1d_1_3(x)
#         x_3 = self.conv_1d_17(x)
#         #print(x_3.shape)
#         x_4 = self.maxpool_1d(x)
#         x_4 = self.conv_1d_1_4(x)
#         #print(x_4.shape)
#         x = torch.cat([x_1,x_2,x_3,x_4], dim = 1)
#         x = self.batch_norm_1d(x)
        
#         x = eval(f'F.{self.activision_function}(x)' )
        
#         return x


# class SkipConnection(nn.Module) :
#     def __init__(self, down_sample, n_filters_in,n_filters_out):
#         super(SkipConnection, self).__init__()
        
#         self.down_sample = down_sample
#         self.n_filters_in = n_filters_in
#         self.n_filters_out = n_filters_out
#         self.maxpool_1d = nn.MaxPool1d(self.down_sample, stride=self.down_sample)
#         self.conv_1d = nn.Conv1d(self.n_filters_in, self.n_filters_out, kernel_size=1, stride=1)
#     def forward(self, y) :
#         if self.down_sample > 1:
#             y = self.maxpool_1d(y)
#         elif self.down_sample == 1:
#             y = y
#         else:
#             raise ValueError("Number of samples should always decrease.")
#         # Deal with n_filters dimension increase
#         if self.n_filters_in != self.n_filters_out:
#             # This is one of the two alternatives presented in ResNet paper
#             # Other option is to just fill the matrix with zeros.
#             y = self.conv_1d(y)
#         return y
    
    
    
# class BatchNorm_PlusActivision(nn.Module) :
#     def __init__(self, n_filters_in, activision_function = 'relu',postactivation_bn= 'True'):
#         super(BatchNorm_PlusActivision, self).__init__()
        
#         self.activision_function = activision_function
#         self.postactivation_bn =   postactivation_bn  
#         self.n_filters_in = n_filters_in
    
#         self.batch_norm_1d =nn.BatchNorm1d(num_features = self.n_filters_in)
#     def forward(self, x) :
#         if self.postactivation_bn : 
#             x = eval(f'F.{self.activision_function}(x)' )
#             x = self.batch_norm_1d(x)
#         else :
#             x = self.batch_norm_1d(x)
#             x = eval(f'F.{self.activision_function}(x)' )
#         return x
    
    
# class ResidualUnit(nn.Module):
#     def __init__(self,down_sample, n_filters_in, n_filters_out, 
#                  dropout_keep_prob=0.8, kernel_size=17, preactivation=True,
#                  postactivation_bn=False, activation_function='relu', last_layer = 'sigmoid'):
#         super(ResidualUnit, self).__init__()
                
#         self.down_sample = down_sample    
#         self.n_filters_in = n_filters_in
#         self.n_filters_out = n_filters_out
#         self.dropout_rate = 1 - dropout_keep_prob
#         self.kernel_size = kernel_size
#         self.preactivation = preactivation
#         self.postactivation_bn = postactivation_bn
#         self.activation_function = activation_function
#         self.last_layer = last_layer
#         self.kernel_size = kernel_size
      
        
        
#         self.conv_1d_1 = nn.Conv1d(self.n_filters_in, self.n_filters_out, kernel_size=self.kernel_size, padding = 8)
#         self.conv_1d_2 = nn.Conv1d(self.n_filters_out, self.n_filters_out, kernel_size=self.kernel_size, stride = self.down_sample,padding = 8)
#         self.skip_connection = SkipConnection(self.down_sample, self.n_filters_in,self.n_filters_out)
#         self.batch_norm_plus_activision = BatchNorm_PlusActivision(self.n_filters_out, self.activation_function, self.postactivation_bn)
#         self.dropout =  nn.Dropout(self.dropout_rate)
        
#     def forward(self, x, y):
#         y = self.skip_connection(y)
#         x = self.conv_1d_1(x)
#         x = self.conv_1d_2(x)
#         if self.dropout_rate > 0 :
#             x = self.dropout(x)
#         if self.preactivation:
#             x = torch.add(x,y)
#             y = x
#             x = self.batch_norm_plus_activision(x)
#             if self.dropout_rate > 0 :
#                  x = self.dropout(x)
#         else:
#             x = self.batch_norm_plus_activision(x)
#             x = torch.add(x,y)
#             x = eval(f'F.{self.activision_function}(x)' )
#             if self.dropout_rate > 0 :
#                     x = self.dropout(x)
#             y = x
#         return x,y
    
    
# class ResidualNetwork(nn.Module) :
#     def __init__(self, n_filters_in,
#                  dropout_keep_prob=0.8, kernel_size=17, preactivation=True,
#                  postactivation_bn=False, activation_function='relu', last_layer = 'sigmoid'):
#         super(ResidualNetwork, self).__init__()
          
#         self.n_filters_in = n_filters_in
#         self.dropout_keep_prob = dropout_keep_prob
#         self.kernel_size = kernel_size
#         self.preactivation = preactivation
#         self.postactivation_bn = postactivation_bn
#         self.activation_function = activation_function
#         self.last_layer = last_layer
#         self.kernel_size = kernel_size
      
        
#         #self.conv_1d = nn.Conv1d(n_filters_in,64, kernel_size=self.kernel_size, padding = 8)
#         #self.batch_norm_1d = nn.BatchNorm1d(num_features = 64)
#         self.inception_unit  =  Inception_module(n_filters_in = self.n_filters_in, n_filters_out = 64, activision_function = 'relu')
#         self.residual_unit_1 = ResidualUnit(4,64,128)
#         self.residual_unit_2 = ResidualUnit(4,128,196)
#         self.residual_unit_3 = ResidualUnit(4,196,256)
#         self.residual_unit_4 = ResidualUnit(4,256,320)
#         self.linear = nn.Linear(5120, 30)
#         self.Sigmoid = nn.Sigmoid()
        
#     def forward(self, x) :
        
#         x = self.inception_unit(x)
#         x = eval(f'F.{self.activation_function}(x)' )
#         x,y = self.residual_unit_1(x,x)
#         x,y = self.residual_unit_2(x,y)
#         x,y = self.residual_unit_3(x,y)
#         x,_ = self.residual_unit_4(x,y)
#         x = x.view(x.size(0),-1)
#         x = self.linear(x)
#         x = self.Sigmoid(x)
#         return x

# Global pool Network

In [9]:
class PcamPool(nn.Module):

    def __init__(self):
        super(PcamPool, self).__init__()

    def forward(self, feat_map, logit_map):
        assert logit_map is not None

        prob_map = torch.sigmoid(logit_map)
        weight_map = prob_map / prob_map.sum(dim=2, keepdim=True)
        feat = (feat_map * weight_map).sum(dim=2, keepdim=True)

        return feat
    
class LogSumExpPool(nn.Module):

    def __init__(self, gamma):
        super(LogSumExpPool, self).__init__()
        self.gamma = gamma

    def forward(self, feat_map):
        """
        Numerically stable implementation of the operation
        Arguments:
            feat_map(Tensor): tensor with shape (N, C, H, W)
            return(Tensor): tensor with shape (N, C, 1, 1)
        """
        (N, C, H,) = feat_map.shape

        # (N, C, 1, 1) m
        m, _ = torch.max(
            feat_map, dim=-1, keepdim=True)[0]

        # (N, C, H, W) value0
        value0 = feat_map - m
        area = 1.0 / (H)
        g = self.gamma

        # TODO: split dim=(-1, -2) for onnx.export
        return m + 1 / g * torch.log(area * torch.sum(
            torch.exp(g * value0), dim=(-1), keepdim=True))

class ExpPool(nn.Module):

    def __init__(self):
        super(ExpPool, self).__init__()

    def forward(self, feat_map):
        """
        Numerically stable implementation of the operation
        Arguments:
            feat_map(Tensor): tensor with shape (N, C, H)
            return(Tensor): tensor with shape (N, C, 1)
        """

        EPSILON = 1e-7
        (N, C, H) = feat_map.shape
        m, _ = torch.max(
            feat_map, dim=-1, keepdim=True)[0]

        # caculate the sum of exp(xi)
        # TODO: split dim=(-1, -2) for onnx.export
        sum_exp = torch.sum(torch.exp(feat_map - m),
                            dim=(-1), keepdim=True)

        # prevent from dividing by zero
        sum_exp += EPSILON

        # caculate softmax in shape of (H,W)
        exp_weight = torch.exp(feat_map - m) / sum_exp
        weighted_value = feat_map * exp_weight

        # TODO: split dim=(-1, -2) for onnx.export
        return torch.sum(weighted_value, dim=(-1), keepdim=True)

class LinearPool(nn.Module):

    def __init__(self):
        super(LinearPool, self).__init__()

    def forward(self, feat_map):
        """
        Arguments:
            feat_map(Tensor): tensor with shape (N, C, H)
            return(Tensor): tensor with shape (N, C, 1)
        """
        EPSILON = 1e-7
        (N, C, H) = feat_map.shape

        # sum feat_map's last two dimention into a scalar
        # so the shape of sum_input is (N,C,1,1)
        # TODO: split dim=(-1, -2) for onnx.export
        sum_input = torch.sum(feat_map, dim=(-1), keepdim=True)

        # prevent from dividing by zero
        sum_input += EPSILON

        # caculate softmax in shape of (H,W)
        linear_weight = feat_map / sum_input
        weighted_value = feat_map * linear_weight

        # TODO: split dim=(-1, -2) for onnx.export
        return torch.sum(weighted_value, dim=(-1), keepdim=True)   
class GlobalPool(nn.Module):

    def __init__(self, pool='PCAM'):
        super(GlobalPool, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.maxpool = nn.AdaptiveMaxPool1d(1)
        self.exp_pool = ExpPool()
        self.pcampool = PcamPool()
        self.linear_pool = LinearPool()
        self.lse_pool = LogSumExpPool(gamma = .9)
        self.pool = pool
    def cuda(self, device=None):
        return self._apply(lambda t: t.cuda(device))

    def forward(self, feat_map, logit_map):
        if self.pool == 'AVG':
            return self.avgpool(feat_map)
        elif self.pool == 'MAX':
            return self.maxpool(feat_map)
        elif self.pool == 'PCAM':
            return self.pcampool(feat_map, logit_map)
        elif self.pool == 'AVG_MAX':
            a = self.avgpool(feat_map)
            b = self.maxpool(feat_map)
            return torch.cat((a, b), 1)
        elif self.pool == 'AVG_MAX_LSE':
            a = self.avgpool(feat_map)
            b = self.maxpool(feat_map)
            c = self.lse_pool(feat_map)
            return torch.cat((a, b, c), 1)
        elif self.pool == 'EXP':
            return self.exp_pool(feat_map)
        elif self.pool == 'LINEAR':
            return self.linear_pool(feat_map)
        elif self.pool == 'LSE':
            return self.lse_pool(feat_map)
        else:
            raise Exception('Unknown pooling type : {}'
                            .format(self.cfg.global_pool))
            

# Models 

In [10]:
class Classifier(nn.Module) :
  def __init__(self, n_class) :
    super(Classifier,self).__init__()

    self.n_class = n_class
    for i in range(self.n_class) :
      setattr(self, 'fc_' + str(i+1), nn.Conv1d(320, 512, kernel_size=17,stride = 17, padding = 1))
    self.linear = nn.Linear(512,1)
    self.sigmoid = nn.Sigmoid()
  def forward(self,x) :
    logits = list()

    for i in range(self.n_class) :
      classifier = getattr(self, 'fc_' + str(i+1))
      logit = classifier(x)
      
      logit = torch.squeeze(logit)
      logit = self.linear(logit)
      logit = self.sigmoid(logit)
      logits.append(logit)
    if logits[0].shape[0] > 1 :
      output = torch.cat(logits,dim = 1)
    else :
       output = torch.cat(logits,dim = 0)
       
    return output


class AG_classifier(nn.Module) :
  def __init__(self, n_class) :
    super(AG_classifier,self).__init__()

    self.n_class = n_class
    for i in range(self.n_class):
        setattr(self,'conv_'+str(i+1), nn.Conv1d(320,128, kernel_size=17,stride = 17, padding = 1))
        setattr(self,'fc_'+str(i+1), nn.Linear(128,1))
    self.input_layer = 128*self.n_class
    self.linear = nn.Linear(self.input_layer,n_class)
    self.sigmoid = nn.Sigmoid()
        
  def forward(self,x):
    logits = list()
    #attentions = list()
    for i in range(self.n_class) :
        
        classifier = getattr(self, 'conv_' + str(i+1))
        logit = classifier(x)
        
        logit = torch.squeeze(logit)
      
        attention_ = getattr(self, 'fc_'+str(i+1))
        attention_ = self.sigmoid(logit)
        logit = torch.mul(logit,attention_)
        logits.append(logit)
    
    output = torch.cat(logits,dim = 1)                        
    output = self.linear(output)                        
    return output
    
class Classifier_Pool(nn.Module) :
    def __init__(self, n_class) :
        super(Classifier_Pool,self).__init__()
        self.n_class = n_class
        for i in range(self.n_class):
            setattr(self,'conv_'+str(i+1), nn.Conv1d(320,1, kernel_size=1,stride = 1, padding = 0))
            
        self.input_layer = 128*self.n_class
        self.linear = nn.Linear(self.input_layer,n_class)
        self.sigmoid = nn.Sigmoid()
        self.global_pool = GlobalPool(pool = 'PCAM')
        
    def forward(self,feat_map):
        logits = list()
        logit_maps = list()
        #attentions = list()
        for i in range(self.n_class) :
        
            classifier = getattr(self, 'conv_' + str(i+1))
            logit_map = None
            logit_map = classifier(feat_map)
            #print(logit_map.shape)
            logit_maps.append(logit_map.squeeze())
            feat = self.global_pool(feat_map, logit_map)
            logit = classifier(feat)
            logit = logit.squeeze(-1)
            #feat = F.dropout(feat, p=self.fc_drop, training=self.training)
            logit = self.sigmoid(logit) 
            logits.append(logit)
            
        if logits[0].shape[0] == 1 :
            output = torch.cat(logits,dim = 0).t() 
        else :
            output = torch.cat(logits,dim = 1)                        
        return output,logit_maps
        
        
        
        
    
class SkipConnection(nn.Module) :
    def __init__(self, down_sample, n_filters_in,n_filters_out):
        super(SkipConnection, self).__init__()
        
        self.down_sample = down_sample
        self.n_filters_in = n_filters_in
        self.n_filters_out = n_filters_out
        self.maxpool_1d = nn.MaxPool1d(self.down_sample, stride=self.down_sample)
        self.conv_1d = nn.Conv1d(self.n_filters_in, self.n_filters_out, kernel_size=1, stride=1)
    def forward(self, y) :
        if self.down_sample > 1:
            y = self.maxpool_1d(y)
        elif self.down_sample == 1:
            y = y
        else:
            raise ValueError("Number of samples should always decrease.")
        # Deal with n_filters dimension increase
        if self.n_filters_in != self.n_filters_out:
            # This is one of the two alternatives presented in ResNet paper
            # Other option is to just fill the matrix with zeros.
            y = self.conv_1d(y)
        return y
    
    
    
class BatchNorm_PlusActivision(nn.Module) :
    def __init__(self, n_filters_in, activision_function = 'relu',postactivation_bn = 'True'):
        super(BatchNorm_PlusActivision, self).__init__()
        
        self.activision_function = activision_function
        self.postactivation_bn =   postactivation_bn  
        self.n_filters_in = n_filters_in
    
        self.batch_norm_1d =nn.BatchNorm1d(num_features = self.n_filters_in)
    def forward(self, x) :
        if self.postactivation_bn : 
            x = eval(f'F.{self.activision_function}(x)' )
            x = self.batch_norm_1d(x)
        else :
            x = self.batch_norm_1d(x)
            x = eval(f'F.{self.activision_function}(x)' )
        return x
    
    
class ResidualUnit(nn.Module):
    def __init__(self,down_sample, n_filters_in, n_filters_out, 
                 dropout_keep_prob=0.8, kernel_size=17, preactivation=True,
                 postactivation_bn=False, activation_function='relu', last_layer = 'sigmoid'):
        super(ResidualUnit, self).__init__()
                
        self.down_sample = down_sample    
        self.n_filters_in = n_filters_in
        self.n_filters_out = n_filters_out
        self.dropout_rate = 1 - dropout_keep_prob
        self.kernel_size = kernel_size
        self.preactivation = preactivation
        self.postactivation_bn = postactivation_bn
        self.activation_function = activation_function
        self.last_layer = last_layer
        self.kernel_size = kernel_size
      
        
        
        self.conv_1d_1 = nn.Conv1d(self.n_filters_in, self.n_filters_out, kernel_size=self.kernel_size, padding = 8)
        self.conv_1d_2 = nn.Conv1d(self.n_filters_out, self.n_filters_out, kernel_size=self.kernel_size, stride = self.down_sample,padding = 8)
        self.skip_connection = SkipConnection(self.down_sample, self.n_filters_in,self.n_filters_out)
        self.batch_norm_plus_activision = BatchNorm_PlusActivision(self.n_filters_out, self.activation_function, self.postactivation_bn)
        self.dropout =  nn.Dropout(self.dropout_rate)
        
    def forward(self, x, y):
        y = self.skip_connection(y)
        x = self.conv_1d_1(x)
        x = self.conv_1d_2(x)
        if self.dropout_rate > 0 :
            x = self.dropout(x)
        if self.preactivation:
            x = torch.add(x,y)
            y = x
            x = self.batch_norm_plus_activision(x)
            if self.dropout_rate > 0 :
                 x = self.dropout(x)
        else:
            x = self.batch_norm_plus_activision(x)
            x = torch.add(x,y)
            x = eval(f'F.{self.activision_function}(x)' )
            if self.dropout_rate > 0 :
                    x = self.dropout(x)
            y = x
        return x,y
    
    
class ResidualNetwork_classifier(nn.Module) :
    def __init__(self, n_filters_in,
                 dropout_keep_prob=0.8, kernel_size=17, preactivation=True,
                 postactivation_bn=False, activation_function='relu', last_layer = 'sigmoid'):
        super(ResidualNetwork_classifier, self).__init__()
          
        self.n_filters_in = n_filters_in
        self.dropout_keep_prob = dropout_keep_prob
        self.kernel_size = kernel_size
        self.preactivation = preactivation
        self.postactivation_bn = postactivation_bn
        self.activation_function = activation_function
        self.last_layer = last_layer
        self.kernel_size = kernel_size
      
        
        self.conv_1d = nn.Conv1d(n_filters_in,64, kernel_size=self.kernel_size, padding = 8)
        self.batch_norm_1d = nn.BatchNorm1d(num_features = 64)
        self.residual_unit_1 = ResidualUnit(4,64,128)
        self.residual_unit_2 = ResidualUnit(4,128,196)
        self.residual_unit_3 = ResidualUnit(4,196,256)
        self.residual_unit_4 = ResidualUnit(4,256,320)
        self.classifier = Classifier_Pool(30)
        #self.linear = nn.Linear(5120, 9)
        #self.Sigmoid = nn.Sigmoid()
        
    def forward(self, x) :
        
        x = self.conv_1d(x)
        x = self.batch_norm_1d(x)
        x = eval(f'F.{self.activation_function}(x)' ) 
        x,y = self.residual_unit_1(x,x)
        x,y = self.residual_unit_2(x,y)
        x,y = self.residual_unit_3(x,y)
        x,_ = self.residual_unit_4(x,y)
        #print(x.shape)
        x,logit_maps = self.classifier(x)
        
        return x,logit_maps
    


In [11]:
model = ResidualNetwork_classifier(12)
a= torch.rand((64,12,4096))
x,y = model(a)
len(y)

30

In [12]:
c = []
for i in range(len(y)):
    temp = []
    for j in range(len(y)):
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        x = cos(y[i],y[j])
        c.append(x)

In [13]:
l = torch.tensor([[1,0,0,0,1],[1,0,1,0,0],[0,0,1,1,0]])
p = []
for i in range(len(l)):
    for j in range(len(l):
        x = torch
        p.append(x)
q  = torch.Tensor(3,5,5)
r = torch.cat(p, out=q)
p.shape

SyntaxError: invalid syntax (<ipython-input-13-33138a53790f>, line 4)

In [ ]:
l[0].unsqueeze(dim = 0)

In [ ]:
torch.mm(l[0].unsqueeze(dim = 0).T,l[0].unsqueeze(dim = 0))

# EXP

In [ ]:
c = []
for i in range(len(x)):
    temp = []
    for j in range(len(x)):
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        z = cos(x[i],x[j])
        c.append(z)
e = torch.stack(c, dim =0)
e = e.reshape(30,30,128)
e = torch.permute(e,(2,0,1))

In [ ]:
l = b
p = []
for i in range(len(l)):

    expand = torch.mm(l[i].unsqueeze(dim = 1),l[i].unsqueeze(dim = 0))
    p.append(expand)

d = torch.stack(p, dim =0)
n = d.shape[1]
mask = torch.eye(n, n).bool()
d = d[:].masked_fill_(mask, False)
r = d*e

In [ ]:
model = ResidualNetwork_classifier(12)
a,b,_ =next(itter(

In [ ]:
a = [torch.rand(64,1,16),torch.rand(64,1,16) ,torch.rand(64,1,16) ,torch.rand(64,1,16) ,torch.rand(64,1,16) ] 
c = []

for i in range(len(a)):
    temp = []
    for j in range(len(a)):
        cos = nn.CosineSimilarity(dim=2, eps=1e-6)
        x = cos(a[i],a[j])
        c.append(x)
    
    
    

In [18]:
print(len(c))
print(c[0].shape)

25
torch.Size([64, 1])


In [21]:
b = torch.Tensor(25,64,1)
d = torch.cat(c, out = b)
d = d.reshape(5,5,64,1)
d.squeeze().shape


torch.Size([5, 5, 64])

In [76]:
l = torch.tensor([[1,0,0,0,1],[1,0,0,0,1],[1,0,0,0,1],[1,0,0,0,1]])
p = []
for i in range(len(l)):

    expand = torch.mm(l[i].unsqueeze(dim = 1),l[i].unsqueeze(dim = 0))
    p.append(expand)

d = torch.stack(p, dim =0)
n = d.shape[1]
mask = torch.eye(n, n).bool()
d[:].masked_fill_(mask, False)

tensor([[[0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0]]])

In [71]:
ind = torch.diag_indices(d[0].shape[0])
d[:,ind[0],ind[1]] = 0
d

AttributeError: module 'torch' has no attribute 'diag_indices'

In [106]:
~torch.tensor([1,0,0,0,1])+2

tensor([0, 1, 1, 1, 0])

In [34]:
b = torch.Tensor(25,64,1)
d = torch.cat(c, out = b)
d = d.reshape(5,5,64,1)
d.squeeze().shape

25

In [73]:
mask = torch.eye(5, 5).byte()
mask

tensor([[1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1]], dtype=torch.uint8)

In [121]:
a.shape

torch.Size([1, 12, 4096])

# Training and testing loop

In [2]:

def cosine_similarityloss(logit_maps,labels) :
    cosine_similarity_list = []
    for i in range(len(logit_maps)):
        for j in range(len(logit_maps)):
            cos = nn.CosineSimilarity(dim=1, eps=1e-6)
            c_similarity = cos(logit_maps[i],logit_maps[j])
            #print(c_similarity.shape)
            cosine_similarity_list.append(c_similarity)
    cosine_similarity_tensor = torch.stack(cosine_similarity_list, dim =0)
    cosine_similarity_tensor = cosine_similarity_tensor.reshape(30,30,128)
    cosine_similarity_tensor = torch.permute(cosine_similarity_tensor,(2,0,1))
    
    ##Masking_tensor
    
    masking_list_cc = []
    masking_list_ca = []
    for i in range(len(labels)):
        inverted_labels = ~labels[i].unsqueeze(dim = 0).int()+2
        inverted_labels = inverted_labels.double()
        expand_cc = torch.mm(labels[i].unsqueeze(dim = 1),labels[i].unsqueeze(dim = 0))
        expand_ca = torch.mm(labels[i].unsqueeze(dim = 1),inverted_labels)
        masking_list_cc.append(expand_cc)
        masking_list_ca.append(expand_ca)

    masking_tensor_cc = torch.stack(masking_list_cc, dim =0)
    masking_tensor_ca = torch.stack(masking_list_ca, dim =0)
    
    n = masking_tensor_cc.shape[1]
    mask = torch.eye(n, n).bool()
    masking_tensor_cc = masking_tensor_cc[:].masked_fill_(mask, 0)
    masking_tensor_ca = masking_tensor_ca[:].masked_fill_(mask, 0)
    
    loss = (.7*(masking_tensor_cc*cosine_similarity_tensor).sum()+.3*(masking_tensor_ca*cosine_similarity_tensor).sum())
    return loss

In [35]:
a  = torch.rand(16,10,10)
logit_maps = [a,a,a,a,a]


n_class = 5


In [34]:
#a = cosine_similarityloss(logit_maps,None,5)


In [251]:
num_epochs = 1000
learning_rate = 0.001
resume = None
model = ResidualNetwork_classifier(12).cuda() 
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
Lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer , step_size = 5, gamma = .1)
label_directory = 'test_data'
output_directory = 'test_outputs'
start_epoch = 0
if resume:
    if os.path.isfile(resume):
      print_log("=> loading checkpoint '{}'".format(resume), log)
      checkpoint = torch.load(resume)
      start_epoch = checkpoint['epoch']
      Lr_scheduler_state_dict = checkpoint['Lr_scheduler_state_dict']
      model.load_state_dict(checkpoint['model_state_dict'])
      optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
      print_log("=> loaded checkpoint '{}' (epoch {})" .format(resume, checkpoint['epoch']), log)
    else:
      print_log("=> no checkpoint found at '{}'".format(resume), log)
Lr_scheduler.step()
for epoch in range(start_epoch,num_epochs):
        since = time.time()
        print('Epoch {}/{}'.format(epoch , num_epochs - 1))
        print('-' * 10)
    
        model.train() 
        running_loss = 0.0

        for i, (inputs, target, header_files) in tqdm.tqdm(enumerate(train_loader)):
            
            input_var = torch.autograd.Variable(inputs.cuda().float())
            target_var = torch.autograd.Variable(target.cuda().float())
            optimizer.zero_grad()

            # compute output
            output = model(input_var)
            # loss
            loss = criterion(output, target_var)
#             print(output)
#             print(target_var)
        
            if (i%5) == 0: 
                print('step: {} totalloss: {loss:.3f} '.format(i, loss = loss))
                
            loss.backward() 
            optimizer.step()  
            

            #print(loss.data.item()
            
            
            
            running_loss += loss.data.item()

                
            
        Lr_scheduler.step()
        epoch_loss = float(running_loss) / float(i)
        print(' Epoch over  Loss: {:.5f}'.format(epoch_loss))
        print('Testing_model............')
        auroc,auc  = test_model(model, classes, test_loader, label_directory, output_directory)
        PATH = f'model/Epoch-{epoch}-aur-{round(auroc,3)}-auc {round(auc,3)}.tar' 
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            'Lr_scheduler_state_dict' :Lr_scheduler.state_dict()
            }, PATH)
        
        

C:\Users\USER\Anaconda3\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
0it [00:00, ?it/s]

Epoch 0/999
----------


0it [00:02, ?it/s]


KeyboardInterrupt: 

In [ ]:
start_epoch = 5
num_epochs = 10
for epoch in range(start_epoch,num_epochs) :
    print(epoch)

In [ ]:
 auroc,auc  = test_model(model, classes, test_loader, label_directory, output_directory)
    